In [8]:
from manim import *
from numpy import linalg as npl
import jupyter_capture_output

video_scene = " -v WARNING --progress_bar None --disable_caching veff_Scene"
video2_scene = " -v WARNING --progress_bar None --disable_caching veff2_Scene"
image_scene = f" -v WARNING --progress_bar None --disable_caching -r {2*427},{2*240}  -s veff_Scene"

In [9]:
class EffectivePotential(Mobject):
    def __init__(self, center, x_range, y_range, x_length, y_length, params, **kwargs):
        super().__init__(**kwargs)

        self.center = center
        self.x_range = x_range
        self.y_range = y_range
        self.x_length = x_length
        self.y_length = y_length
        self.alpha = params[0]
        self.mass =params[1] 
        self.L = params[2]
        self.E = params[3]

        # coordinate system
        self.ax = Axes(
            x_range = self.x_range, y_range = self.y_range, x_length = self.x_length, y_length = self.y_length, 
            axis_config = {"stroke_color": BLACK, "tip_width": 0.15, "tip_height": 0.15, "include_ticks": False},
            ).move_to(self.center).set_opacity(0.5)
        self.ax_xlabel = self.ax.get_x_axis_label(Tex(r"$r$", font_size = 28, color = BLACK)).shift(0.1 *LEFT)
        self.ax_ylabel = self.ax.get_y_axis_label(Tex(r"$V$", font_size = 28, color = BLACK)).shift(0.15 * DOWN)
        self.add(self.ax, self.ax_xlabel, self.ax_ylabel)


    # returns the function for the central potential
    def central_potential(self, r):
        return -self.alpha / r
    

    # returns the effective potential for given angular momentum L
    def effective_potential(self, r):
        V = self.central_potential(r)
        return V + self.L**2 / (2*self.mass*r**2)
    

    # returns the minimum of the effective potential
    def get_mimimum(self):
        return self.L**2 / (self.mass*self.alpha)
    

    # returns the minimal radius
    def get_r_min(self):
        r = 0.1
        E_eff = self.effective_potential(r)
        while (E_eff > self.E):
            r += 0.0001
            E_eff = self.effective_potential(r)
        return r    
    

    # returns the maximum radius
    def get_r_max(self):
        r = self.get_r_min()
        E_eff = self.effective_potential(r)
        while (E_eff < self.E):
            r += 0.0001
            E_eff = self.effective_potential(r)
        return r
    

    # returns lines for min and max radius
    def get_r_lines(self):
        r_min = self.get_r_min()
        r_max = self.get_r_max()

        E_eff_r_min = self.effective_potential(r_min)
        E_eff_r_max = self.effective_potential(r_max)

        r_min_line = Line(start = self.ax.c2p(r_min, E_eff_r_min, 0), end = self.ax.c2p(r_min, 0.15, 0), color = LIGHT_GRAY, stroke_width = 1)
        r_max_line = Line(start = self.ax.c2p(r_max, E_eff_r_max, 0), end = self.ax.c2p(r_max, 0.15, 0), color = LIGHT_GRAY, stroke_width = 1)

        r_min_descriptor = Tex(r"$r_\mathrm{min}$", font_size = 28, color = GREY).move_to(self.ax.c2p(r_min, 0.5, 0)).shift(0.25*RIGHT)
        r_max_descriptor = Tex(r"$r_\mathrm{max}$", font_size = 28, color = GREY).move_to(self.ax.c2p(r_max, 0.5, 0)).shift(0.25*RIGHT)

        return VGroup(r_min_line, r_max_line, r_min_descriptor, r_max_descriptor)
    

    # returns the minimal radius as the initial condition
    def get_init_state(self):
        r_min = self.get_r_min()
        r_max = self.get_r_max()
        return np.array([r_max, 0])
    

    # returns the next state of a given state in the effective potential (velocity verlet)
    def get_next_state(self, state):
        # algorithm control parameters
        delta_r = 0.001
        delta_t = 0.02

        r = state[0]
        d_r = state[1]
        dd_r = (self.effective_potential(r-delta_r) - self.effective_potential(r+delta_r)) / (self.mass * 2*delta_r)

        a1 = np.array([d_r, dd_r])
        state[0] += a1[0]*delta_t + a1[1]*delta_t**2 / 2

        r = state[0]
        dd_r = (self.effective_potential(r-delta_r) - self.effective_potential(r+delta_r)) / (self.mass * 2*delta_r)

        a2 = np.array([d_r, dd_r])
        state[1] += (a1[1] + a2[1]) * delta_t / 2
        return state
    

    # returns the effective potential plot
    def draw_effective_potential(self):
        x = 0.1
        y = self.effective_potential(x)
        y_max = self.y_range[1]
        while (y > y_max):
            x += 0.005
            y = self.effective_potential(x)
        ax_plot = self.ax.plot(self.effective_potential, x_range = [x, self.x_range[1]], color = BLACK)
        ax_descriptor = Tex(r"$V_\mathrm{eff}$", font_size = 28, color = BLACK).move_to(self.ax.c2p(x, y, 0)).shift(0.5*DOWN + 0.5*RIGHT)
        return VGroup(ax_plot, ax_descriptor)
    

    # draws the line for the total energy of the system
    def draw_energy(self):
        ax_plot = self.ax.plot(lambda x: self.E, x_range = [-0.1, self.x_range[1]], color = PURE_RED).set_opacity(0.5)
        ax_descriptor = Tex(r"$E$", font_size = 28, color = PURE_RED).move_to(self.ax.c2p(-0.5, self.E, 0)).set_opacity(0.75)
        return VGroup(ax_plot, ax_descriptor)
    

    # draws the point on the potential for a given state
    def draw_point(self, state):
        r = state[0]
        veff_r = self.effective_potential(r)
        state_circle = Circle(radius = 0.1, color = PURE_RED, fill_opacity = 0.5).move_to(self.ax.c2p(r, veff_r, 0))
        return state_circle
    


class ConicTrajectory(Mobject):
    def __init__(self, center, x_range, y_range, x_length, y_length, p, epsilon, params, **kwargs):
        super().__init__(**kwargs)

        self.center = center
        self.x_range = x_range
        self.y_range = y_range
        self.x_length = x_length
        self.y_length = y_length
        # self.alpha = alpha
        # self.mass = mass
        # self.p = L**2/(alpha*mass)
        # self.epsilon = np.sqrt( (2*E*L**2) / (alpha**2*mass) )
        self.p = p
        self.epsilon = epsilon
        self.r_min = p / (1 + epsilon)
        self.r_max = p / (1 - epsilon)

        # just in case
        self.alpha = params[0]
        self.mass =params[1] 
        self.L = params[2]
        self.E = params[3]


        # coordinate system
        self.ax = Axes(
            x_range = self.x_range, y_range = self.y_range, x_length = self.x_length, y_length = self.y_length, 
            axis_config = {"stroke_color": BLACK, "tip_width": 0.15, "tip_height": 0.15, "include_ticks": False},
            ).move_to(self.center).set_opacity(0.5)
        # self.add(self.ax)
        sun = Dot(point = self.ax.c2p(0, 0, 0), color = BLACK, radius = 0.1)
        self.add(sun)


    # add radius circles
    def get_r_circles(self):
        r_min_top = self.ax.c2p(0, self.r_min, 0)
        r_min_left = self.ax.c2p(-self.r_min, 0, 0)
        r_min_right = self.ax.c2p(self.r_min, 0, 0)
        inner_circle = Circle.from_three_points(r_min_left, r_min_top, r_min_right, color = LIGHT_GRAY, stroke_width = 1)

        r_min_ax = (self.ax.c2p(self.r_min, 0, 0) - self.ax.c2p(0, 0, 0))[0]
        r_max_ax = (self.ax.c2p(self.r_max, 0, 0) - self.ax.c2p(0, 0, 0))[0]

        outer_circle_dangle = PI / 4
        outer_circle = Arc(radius = r_max_ax, start_angle = PI-outer_circle_dangle, angle = 2*outer_circle_dangle, arc_center = self.ax.c2p(0, 0, 0), color = LIGHT_GRAY, stroke_width = 1)

        r_min_descriptor = Tex(r"$r_\mathrm{min}$", font_size = 28, color = GREY).move_to(self.ax.c2p(-r_min_ax, 0, 0)).shift(0.5*LEFT)
        r_max_descriptor = Tex(r"$r_\mathrm{max}$", font_size = 28, color = GREY).move_to(self.ax.c2p(-r_max_ax, 0, 0)).shift(1.15*LEFT)
        
        return VGroup(inner_circle, outer_circle, r_min_descriptor, r_max_descriptor)


    # analytical solution r(phi) of the 2-body-problem
    def get_r(self, phi):
        return self.p / (1 + self.epsilon*np.cos(phi))
    

    # reverse function phi(r) of the analytical 2-body-problem solution
    def get_phi(self, r):
        dubious_correction_offset = 0.00001
        if r >= self.p / (1-self.epsilon)+dubious_correction_offset:
            return np.arccos( self.p / (1-self.epsilon) - dubious_correction_offset)
        elif r <= self.p / (1+self.epsilon)-dubious_correction_offset:
            return np.arccos( self.p / (1+self.epsilon) + dubious_correction_offset)
        else:
            return np.arccos( (self.p/r - 1) / self.epsilon)


    # draw r(phi) analytical solution
    def draw_trajectory(self):
        phi_array = np.linspace(0, 2*PI, 1000)
        r_array = np.array([self.get_r(phi) for phi in phi_array])

        x_vals = [r_array[i] * np.cos(phi_array[i]) for i in range(len(phi_array))]
        y_vals = [r_array[i] * np.sin(phi_array[i]) for i in range(len(phi_array))]

        r_phi_plot = self.ax.plot_line_graph(x_vals, y_vals, stroke_color = PURE_RED, add_vertex_dots = False, stroke_opacity = 0.5, stroke_width = 4)
        return r_phi_plot
    

    # draw a dot on the analytical trajectory
    def draw_point(self, state):
        r = state[0]
        d_r = state[1]
        if d_r > 0:
            phi = self.get_phi(r)
        else:
            phi = 2*PI - self.get_phi(r)
        x = r * np.cos(phi)
        y = r * np.sin(phi)
        planet = Dot(point = self.ax.c2p(x, y, 0), color = PURE_RED, radius = 0.1)
        return planet
    

    # draw r_velocity vector
    def draw_r_velocity(self, state):
        r = state[0]
        d_r = state[1]
        if d_r > 0:
            phi = self.get_phi(r)
        else:
            phi = 2*PI - self.get_phi(r)
        x = r * np.cos(phi)
        y = r * np.sin(phi)

        dr_x = d_r/3 * np.cos(phi)
        dr_y = d_r/3 * np.sin(phi)

        vector_start = self.ax.c2p(x, y, 0)
        vector_end = self.ax.c2p(x + dr_x, y + dr_y, 0)
        vector = Line(start = vector_start, end = vector_end, color = BLACK).add_tip(tip_length = 0.15, tip_width = 0.15)
        vector.value = d_r
        return vector
    

    # draw phi_velocity vector
    def draw_phi_velocity(self, state):
        r = state[0]
        d_r = state[1]
        if d_r > 0:
            phi = self.get_phi(r)
        else:
            phi = 2*PI - self.get_phi(r)
        x = r * np.cos(phi)
        y = r * np.sin(phi)

        d_phi = self.L / self.mass / r**2

        dphi_x = d_phi/3 * -np.sin(phi)
        dphi_y = d_phi/3 * np.cos(phi)

        vector_start = self.ax.c2p(x, y, 0)
        vector_end = self.ax.c2p(x + dphi_x, y + dphi_y, 0) 
        vector = Line(start = vector_start, end = vector_end, color = GRAY).add_tip(tip_length = 0.15, tip_width = 0.15)
        vector.value = d_phi
        return vector
    

    # draw the total velocity
    def draw_velocity(self, state):
        r = state[0]
        d_r = state[1]
        if d_r > 0:
            phi = self.get_phi(r)
        else:
            phi = 2*PI - self.get_phi(r)
        x = r * np.cos(phi)
        y = r * np.sin(phi)

        d_phi = self.L / self.mass / r**2

        dr_x = d_r/3 * np.cos(phi)
        dr_y = d_r/3 * np.sin(phi)
        dphi_x = d_phi/3 * -np.sin(phi)
        dphi_y = d_phi/3 * np.cos(phi)

        vector_start = self.ax.c2p(x, y, 0)
        vector_end = self.ax.c2p(x + dr_x + dphi_x, y + dr_y + dphi_y, 0) 
        vector = Line(start = vector_start, end = vector_end, color = PURE_RED).add_tip(tip_length = 0.15, tip_width = 0.15)
        vector.value = d_phi

        r_connector_start = self.ax.c2p(x + dr_x, y + dr_y, 0)
        phi_connector_start = self.ax.c2p(x + dphi_x, y + dphi_y, 0)
        r_connector = Line(start = r_connector_start, end = vector_end, color = GRAY, stroke_width = 1)
        phi_connector = Line(start = phi_connector_start, end = vector_end, color = BLACK, stroke_width = 1)
        return VGroup(r_connector, phi_connector, vector)
    


class AxVelocity(Mobject):
    def __init__(self, center, x_range, y_range, x_length, y_length, conic_trajectory, **kwargs):
        super().__init__(**kwargs)

        self.center = center
        self.x_range = x_range
        self.y_range = y_range
        self.x_length = x_length
        self.y_length = y_length

        self.conic_trajectory = conic_trajectory

        # coordinate system
        self.ax = Axes(
            x_range = self.x_range, y_range = self.y_range, x_length = self.x_length, y_length = self.y_length, 
            axis_config = {"stroke_color": BLACK, "tip_width": 0.15, "tip_height": 0.15, "include_ticks": False},
            ).move_to(self.center).set_opacity(0.5)
        self.ax_xlabel = self.ax.get_x_axis_label(Tex(r"$\varphi$", font_size = 28, color = BLACK)).shift(0.1 *LEFT)
        self.ax_ylabel = self.ax.get_y_axis_label(Tex(r"$v$", font_size = 28, color = BLACK)).shift(0.15 * DOWN + 0.5*LEFT)
        self.add(self.ax, self.ax_xlabel, self.ax_ylabel)


    # draws dot for r-velocity
    def get_r_velocity_dot(self, state):
        r = state[0]
        d_r = state[1]
        if d_r > 0:
            phi = self.conic_trajectory.get_phi(r)
        else:
            phi = 2*PI - self.conic_trajectory.get_phi(r)
        d_r_vector = self.conic_trajectory.draw_r_velocity(state).value
        dot = Dot(point = self.ax.c2p(phi, d_r_vector), color = BLACK, radius = 0.05)
        return dot
    

    # draws dot for r-velocity
    def get_phi_velocity_dot(self, state):
        r = state[0]
        d_r = state[1]
        if d_r > 0:
            phi = self.conic_trajectory.get_phi(r)
        else:
            phi = 2*PI - self.conic_trajectory.get_phi(r)
        d_phi_vector = self.conic_trajectory.draw_phi_velocity(state).value
        dot = Dot(point = self.ax.c2p(phi, d_phi_vector), color = GREY, radius = 0.05)
        return dot
    

    # returns ax-points (phi, v_r) and (phi, v_phi)
    def get_ax_points(self, state):
        r = state[0]
        d_r = state[1]
        if d_r > 0:
            phi = self.conic_trajectory.get_phi(r)
        else:
            phi = 2*PI - self.conic_trajectory.get_phi(r) 
        d_r_vector = self.conic_trajectory.draw_r_velocity(state).value
        d_phi_vector = self.conic_trajectory.draw_phi_velocity(state).value
        return (self.ax.c2p(phi, d_r_vector), self.ax.c2p(phi, d_phi_vector))


    # # connects two dots
    # def get_v_connector(self, old_state, new_state):
    #     old_r = old_state[0]
    #     old_d_r = old_state[1]
    #     if old_d_r > 0:
    #         old_phi = self.conic_trajectory.get_phi(old_r)
    #     else:
    #         old_phi = 2*PI - self.conic_trajectory.get_phi(old_r)

In [10]:
%%manim -qh --fps 60 $video_scene


class veff_Scene(ThreeDScene):
    def construct(self):
        self.camera.background_color = WHITE

        # physical parameters
        alpha = 13
        mass = 1
        L = 4
        E = -3
        params = (alpha, mass, L, E)

        # EFFECTIVE POTENTIAL
        veff_center = np.array([4, 0, 0])
        veff_x_range = [0, 6, 1]
        veff_y_range = [-6, 4, 1]
        veff_x_length = 5
        veff_y_length = 5

        effective_potential = EffectivePotential(veff_center, veff_x_range, veff_y_range, veff_x_length, veff_y_length, params)
        self.add(effective_potential)

        effective_potential_plot = effective_potential.draw_effective_potential()
        self.add(effective_potential_plot)

        effective_potential_r_lines = effective_potential.get_r_lines()
        self.add(effective_potential_r_lines)

        effective_potential_energy_plot = effective_potential.draw_energy()
        self.add(effective_potential_energy_plot)

        init_state = effective_potential.get_init_state()
        state_circle = effective_potential.draw_point(init_state)
        state_circle.getter = effective_potential.draw_point
        state_circle.state = init_state
        state_circle.state_getter = effective_potential.get_next_state
        self.add(state_circle)


        # CONIC TRAJECTORY
        traj_center = np.array([-2, 1.5, 0])
        traj_x_range = [-4, 4, 1]
        traj_y_range = [-4, 4, 1]
        traj_x_length = 6
        traj_y_length = 6

        # analytical parameters from numerical V_eff analysis
        dubious_correction_offset = 0.01
        r_min = effective_potential.get_r_min()
        r_max = effective_potential.get_r_max()
        epsilon = (r_max - r_min) / (r_max + r_min)
        p = r_min * (1 + epsilon)

        conic_trajectory = ConicTrajectory(traj_center, traj_x_range, traj_y_range, traj_x_length, traj_y_length, p, epsilon, params)
        self.add(conic_trajectory)

        conic_trajectory_trajectory = conic_trajectory.draw_trajectory()
        self.add(conic_trajectory_trajectory)

        conic_trajectory_r_circles = conic_trajectory.get_r_circles()
        self.add(conic_trajectory_r_circles)

        conic_trajectory_circle = conic_trajectory.draw_point(init_state)
        self.add(conic_trajectory_circle)
        
        conic_trajectory_r_velocity_vector = conic_trajectory.draw_r_velocity(init_state)
        self.add(conic_trajectory_r_velocity_vector)

        conic_trajectory_phi_velocity_vector = conic_trajectory.draw_phi_velocity(init_state)
        self.add(conic_trajectory_phi_velocity_vector)


        # VELOCITY OVER PHI
        axv_center = np.array([-3, -1.5, 0])
        axv_x_range = [0, 2*PI, 0.5*PI]
        axv_y_range = [-4, 10, 1]
        axv_x_length = 4
        axv_y_length = 2

        ax_velocity = AxVelocity(axv_center, axv_x_range, axv_y_range, axv_x_length, axv_y_length, conic_trajectory)
        self.add(ax_velocity)

        ax_velocity_d_r_dot = ax_velocity.get_r_velocity_dot(init_state)
        self.add(ax_velocity_d_r_dot)

        ax_velocity_d_phi_dot = ax_velocity.get_phi_velocity_dot(init_state)
        self.add(ax_velocity_d_phi_dot)



        # updater
        def eff_pot_updater(dot):
            state = dot.state
            # old dot pos
            dot_pos = ax_velocity.get_ax_points(state)
            old_d_r_dot_pos = dot_pos[0]
            old_d_phi_dot_pos = dot_pos[1]
            new_state = dot.state_getter(state)
            # new dot pos
            dot_pos = ax_velocity.get_ax_points(state)
            new_d_r_dot_pos = dot_pos[0]
            new_d_phi_dot_pos = dot_pos[1]       

            dot_getter = dot.getter
            new_dot = dot_getter(new_state)
            conic_trajectory_circle.become(conic_trajectory.draw_point(new_state))
            conic_trajectory_r_velocity_vector.become(conic_trajectory.draw_r_velocity(new_state))
            conic_trajectory_phi_velocity_vector.become(conic_trajectory.draw_phi_velocity(new_state))
            # update dots
            ax_velocity_d_r_dot.become(ax_velocity.get_r_velocity_dot(new_state))
            ax_velocity_d_phi_dot.become(ax_velocity.get_phi_velocity_dot(new_state))
            # new dot pos
            # print(old_d_r_dot_pos, new_d_r_dot_pos)
            if old_d_r_dot_pos[0] < new_d_r_dot_pos[0]:
                d_r_dot_connector = Line(start = old_d_r_dot_pos, end = new_d_r_dot_pos, color = BLACK, stroke_width = 2)
                d_phi_dot_connector = Line(start = old_d_phi_dot_pos, end = new_d_phi_dot_pos, color = GREY, stroke_width = 2)
                self.add(d_r_dot_connector, d_phi_dot_connector)
            # else:
            #     ax_zero = ax_velocity.ax.c2p(0, 0, 0)[0]
            #     old_d_r_dot_pos[0] = ax_zero
            #     old_d_phi_dot_pos[0] = ax_zero
            #     print(ax_velocity.ax.p2c(old_d_r_dot_pos), ax_velocity.ax.p2c(new_d_r_dot_pos))
            #     d_r_dot_connector = Line(start = old_d_r_dot_pos, end = new_d_r_dot_pos, color = BLACK, stroke_width = 2)
            #     d_phi_dot_connector = Line(start = old_d_phi_dot_pos, end = new_d_phi_dot_pos, color = GREY, stroke_width = 2)
            #     self.add(d_r_dot_connector, d_phi_dot_connector)
            dot.become(new_dot)


        t_tracker = ValueTracker(0)
        self.wait(1.5)
        state_circle.add_updater(eff_pot_updater)
        self.play(t_tracker.animate.set_value(5), rate_func= linear, run_time = 2.5)
        state_circle.remove_updater(eff_pot_updater)
        self.wait(0.5)
        conic_trajectory_velocity_vector = conic_trajectory.draw_velocity(state_circle.state)
        self.play(FadeIn(conic_trajectory_velocity_vector), run_time = 3)
        self.wait(3)
        self.play(FadeOut(conic_trajectory_velocity_vector), run_time = 3)
        self.wait(0.5)
        state_circle.add_updater(eff_pot_updater)
        self.play(t_tracker.animate.set_value(5), rate_func= linear, run_time = 15)
        self.wait(3)
        

Manim Community v0.18.1

In [11]:
%%manim -qh --fps 60 $video2_scene


class veff2_Scene(ThreeDScene):
    def construct(self):
        self.camera.background_color = WHITE

        # physical parameters
        alpha = 13
        mass = 1
        L = 4
        E = 1
        params = (alpha, mass, L, E)

        # EFFECTIVE POTENTIAL
        veff_center = np.array([4, 0, 0])
        veff_x_range = [0, 6, 1]
        veff_y_range = [-6, 4, 1]
        veff_x_length = 5
        veff_y_length = 5

        effective_potential = EffectivePotential(veff_center, veff_x_range, veff_y_range, veff_x_length, veff_y_length, params)
        self.add(effective_potential)

        effective_potential_plot = effective_potential.draw_effective_potential()
        self.add(effective_potential_plot)

        effective_potential_r_lines = effective_potential.get_r_lines()
        self.add(effective_potential_r_lines)

        effective_potential_energy_plot = effective_potential.draw_energy()
        self.add(effective_potential_energy_plot)

        init_state = effective_potential.get_init_state()
        state_circle = effective_potential.draw_point(init_state)
        state_circle.getter = effective_potential.draw_point
        state_circle.state = init_state
        state_circle.state_getter = effective_potential.get_next_state
        self.add(state_circle)


        # CONIC TRAJECTORY
        traj_center = np.array([-2, 1.5, 0])
        traj_x_range = [-4, 4, 1]
        traj_y_range = [-4, 4, 1]
        traj_x_length = 6
        traj_y_length = 6

        # analytical parameters from numerical V_eff analysis
        dubious_correction_offset = 0.01
        r_min = effective_potential.get_r_min()
        r_max = effective_potential.get_r_max()
        epsilon = (r_max - r_min) / (r_max + r_min)
        p = r_min * (1 + epsilon)

        conic_trajectory = ConicTrajectory(traj_center, traj_x_range, traj_y_range, traj_x_length, traj_y_length, p, epsilon, params)
        self.add(conic_trajectory)

        conic_trajectory_trajectory = conic_trajectory.draw_trajectory()
        self.add(conic_trajectory_trajectory)

        conic_trajectory_r_circles = conic_trajectory.get_r_circles()
        self.add(conic_trajectory_r_circles)

        conic_trajectory_circle = conic_trajectory.draw_point(init_state)
        self.add(conic_trajectory_circle)
        
        conic_trajectory_r_velocity_vector = conic_trajectory.draw_r_velocity(init_state)
        self.add(conic_trajectory_r_velocity_vector)

        conic_trajectory_phi_velocity_vector = conic_trajectory.draw_phi_velocity(init_state)
        self.add(conic_trajectory_phi_velocity_vector)


        # VELOCITY OVER PHI
        axv_center = np.array([-3, -1.5, 0])
        axv_x_range = [0, 2*PI, 0.5*PI]
        axv_y_range = [-4, 10, 1]
        axv_x_length = 4
        axv_y_length = 2

        ax_velocity = AxVelocity(axv_center, axv_x_range, axv_y_range, axv_x_length, axv_y_length, conic_trajectory)
        self.add(ax_velocity)

        ax_velocity_d_r_dot = ax_velocity.get_r_velocity_dot(init_state)
        self.add(ax_velocity_d_r_dot)

        ax_velocity_d_phi_dot = ax_velocity.get_phi_velocity_dot(init_state)
        self.add(ax_velocity_d_phi_dot)



        # updater
        def eff_pot_updater(dot):
            state = dot.state
            # old dot pos
            dot_pos = ax_velocity.get_ax_points(state)
            old_d_r_dot_pos = dot_pos[0]
            old_d_phi_dot_pos = dot_pos[1]
            new_state = dot.state_getter(state)
            # new dot pos
            dot_pos = ax_velocity.get_ax_points(state)
            new_d_r_dot_pos = dot_pos[0]
            new_d_phi_dot_pos = dot_pos[1]       

            dot_getter = dot.getter
            new_dot = dot_getter(new_state)
            conic_trajectory_circle.become(conic_trajectory.draw_point(new_state))
            conic_trajectory_r_velocity_vector.become(conic_trajectory.draw_r_velocity(new_state))
            conic_trajectory_phi_velocity_vector.become(conic_trajectory.draw_phi_velocity(new_state))
            # update dots
            ax_velocity_d_r_dot.become(ax_velocity.get_r_velocity_dot(new_state))
            ax_velocity_d_phi_dot.become(ax_velocity.get_phi_velocity_dot(new_state))
            # new dot pos
            # print(old_d_r_dot_pos, new_d_r_dot_pos)
            if old_d_r_dot_pos[0] < new_d_r_dot_pos[0]:
                d_r_dot_connector = Line(start = old_d_r_dot_pos, end = new_d_r_dot_pos, color = BLACK, stroke_width = 2)
                d_phi_dot_connector = Line(start = old_d_phi_dot_pos, end = new_d_phi_dot_pos, color = GREY, stroke_width = 2)
                self.add(d_r_dot_connector, d_phi_dot_connector)
            # else:
            #     ax_zero = ax_velocity.ax.c2p(0, 0, 0)[0]
            #     old_d_r_dot_pos[0] = ax_zero
            #     old_d_phi_dot_pos[0] = ax_zero
            #     print(ax_velocity.ax.p2c(old_d_r_dot_pos), ax_velocity.ax.p2c(new_d_r_dot_pos))
            #     d_r_dot_connector = Line(start = old_d_r_dot_pos, end = new_d_r_dot_pos, color = BLACK, stroke_width = 2)
            #     d_phi_dot_connector = Line(start = old_d_phi_dot_pos, end = new_d_phi_dot_pos, color = GREY, stroke_width = 2)
            #     self.add(d_r_dot_connector, d_phi_dot_connector)
            dot.become(new_dot)


        t_tracker = ValueTracker(0)
        self.wait(1.5)
        state_circle.add_updater(eff_pot_updater)
        self.play(t_tracker.animate.set_value(5), rate_func= linear, run_time = 2.5)
        state_circle.remove_updater(eff_pot_updater)
        self.wait(0.5)
        conic_trajectory_velocity_vector = conic_trajectory.draw_velocity(state_circle.state)
        self.play(FadeIn(conic_trajectory_velocity_vector), run_time = 3)
        self.wait(3)
        self.play(FadeOut(conic_trajectory_velocity_vector), run_time = 3)
        self.wait(0.5)
        state_circle.add_updater(eff_pot_updater)
        self.play(t_tracker.animate.set_value(5), rate_func= linear, run_time = 15)
        self.wait(3)
        

Manim Community v0.18.1

KeyboardInterrupt: 